In [1]:
!pip install catboost

     |████████████████████████████████| 64.8MB 59kB/s 


In [3]:
!pip install unidecode

In [4]:
!pip install category_encoders

     |████████████████████████████████| 81kB 3.0MB/s 


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
import catboost as cb
import lightgbm as lgb
import re
import unidecode
import nltk
from nltk.tokenize import TweetTokenizer
import string
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
import category_encoders as ce

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
id='1cQhZ_2NeVAXqkXpmaCQbJVxLP4b1lNAf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train.csv')

id='12nWwwnj9jbtAsk0VENlQaV3d6jclQ-1S'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test.csv')

In [8]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [9]:
def remove_punctuation(word):        
    clean_word = ''.join([char for char in word if char not in string.punctuation])
    return clean_word

def cleaning_text(text):
    tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
    text_tokenize = tokenizer.tokenize(text)
    wordlist = []
    for word in text_tokenize:
        word = word.lower()
        word = re.sub('(?P<url>https?://[^\s]+)', ' ', word)
        word = remove_punctuation(word)
        word = re.sub(r'[^\w]', ' ', word)
        word = unidecode.unidecode(word)
        word = re.sub(r'[0-9]','', word)
        if((word != '')&(word != ' ')&(word not in stopwords)):
            wordlist.append(word)
    clean_text = ' '.join(wordlist)
    return clean_text

In [10]:
train["clean_text"] = train["text"].apply(lambda x: cleaning_text(x))
test["clean_text"] = test["text"].apply(lambda x: cleaning_text(x))

In [11]:
train["keyword"] = train["keyword"].str.replace('%20',' ')
train["keyword"] = train["keyword"].astype('category')
test["keyword"] = test["keyword"].str.replace('%20',' ')
test["keyword"] = test["keyword"].astype('category')

In [12]:
train.rename(columns={"target":"target_label"}, inplace=True)
train.rename(columns={"location":"location_original"}, inplace=True)
train.rename(columns={"id":"id_original"}, inplace=True)
train.rename(columns={"text":"text_original"}, inplace=True)
train.rename(columns={"keyword":"keyword_original"}, inplace=True)
train.head()

,id_original,keyword_original,location_original,text_original,target_label,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...


In [13]:
# test.rename(columns={"target":"target_label"}, inplace=True)
test.rename(columns={"location":"location_original"}, inplace=True)
test.rename(columns={"id":"id_original"}, inplace=True)
test.rename(columns={"text":"text_original"}, inplace=True)
test.rename(columns={"keyword":"keyword_original"}, inplace=True)
test.head()

,id_original,keyword_original,location_original,text_original,clean_text
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different cities stay safe ev...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan


In [15]:
hashing_encoder = ce.HashingEncoder(cols=['keyword_original'])
hashing_train = hashing_encoder.fit_transform(train['keyword_original'])
train_hash = pd.concat([train, hashing_train], axis=1)
train_hash.head()

,id_original,keyword_original,location_original,text_original,target_label,clean_text,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason earthquake may allah forgive us,1,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,1,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...,1,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders cal...,1,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfires pou...,1,0,0,0,0,0,0,0


In [19]:
hashing_test = hashing_encoder.fit_transform(test['keyword_original'])
test_hash = pd.concat([test, hashing_test], axis=1)
test_hash.head()

,id_original,keyword_original,location_original,text_original,clean_text,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash,1,0,0,0,0,0,0,0
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquake different cities stay safe ev...,1,0,0,0,0,0,0,0
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...,1,0,0,0,0,0,0,0
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires,1,0,0,0,0,0,0,0
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan,1,0,0,0,0,0,0,0


In [23]:
#BOW
vectorizer = CountVectorizer(stop_words='english', binary=True)
vectorizer.fit(train["clean_text"])

vec_train = vectorizer.fit_transform(train["clean_text"])
feature_words = vectorizer.get_feature_names()
print(len(feature_words))
df_bow_train = pd.DataFrame(vec_train.toarray(), columns=feature_words)
df_bow_train.head()

14190


,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
vec_test = vectorizer.transform(test["clean_text"])
df_bow_test = pd.DataFrame(vec_test.toarray(), columns=feature_words)
df_bow_test.head()

,aa,aaa,aaaand,aaalll,aaarrrgghhh,aaemiddleaged,aal,aan,aannnd,aar,aashiqui,ab,aba,abandon,abandoned,abandoning,abbandoned,abbott,abbruchsimulator,abbswinston,abc,abceyewitness,abcnews,abcs,abe,aberdeen,aberystwythshrewsbury,abes,abha,abia,ability,abject,ablaze,able,ableg,aboard,abomb,abombed,abomination,abortion,...,zayn,zeal,zehrs,zenit,zergele,zero,zeros,zhejiang,zicac,zimbabwe,zimmer,zimmerman,zimpapersviews,zionism,zionist,zionists,zip,ziphimup,zipped,zippednews,zipper,zippers,zippoline,ziuw,zmne,zodiac,zombie,zombiefunrun,zombies,zone,zones,zonewolf,zoom,zotar,zouma,zrnf,zss,zumiez,zurich,zzz
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
print(df_bow_train.shape)
print(df_bow_test.shape)

(7613, 14190)
(3263, 14190)


In [ ]:
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>1)]
train_final = pd.concat([train_hash,filter_train], axis="columns")
print(train_final.shape)
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>=5)]
train_final_2 = pd.concat([train_hash,filter_train], axis="columns")
print(train_final_2.shape)
filter_train = df_bow_train.loc[:,(df_bow_train.sum()>=10)]
train_final_3 = pd.concat([train_hash,filter_train], axis="columns")
print(train_final_3.shape)

(7613, 6063)
(7613, 2505)
(7613, 1337)


In [ ]:
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>1)]
# test_final = pd.concat([test_hash,filter_test], axis="columns")
# print(test_final.shape)
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>=5)]
# test_final_2 = pd.concat([test_hash,filter_test], axis="columns")
# print(test_final_2.shape)
# filter_test = df_bow_test.loc[:,(df_bow_test.sum()>=10)]
# test_final_3 = pd.concat([test_hash,filter_test], axis="columns")
# print(test_final_3.shape)

In [ ]:
# feature selection
def select_features(X_train, y_train, X_test, function, k):
    fs = SelectKBest(score_func=function, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs

In [ ]:
X = train_final_2.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_final_2["target_label"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 2499)
(5709,)
(1904, 2499)
(1904,)


## Usando 2499 features

In [ ]:
model = xgb.XGBClassifier(n_estimators=500, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.803571


In [ ]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.799370


In [ ]:
model = cb.CatBoostClassifier(iterations=500, depth=7, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

In [ ]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.798845


In [ ]:
model = KNeighborsClassifier(n_neighbors=11, metric="jaccard")
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.775735


## Usando 300 features

In [ ]:
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 300)

In [ ]:
model = xgb.XGBClassifier(n_estimators=500, objective='binary:logistic', max_depth=9, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.782038


In [ ]:
model = lgb.LGBMClassifier(num_leaves=21, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=9, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=9,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=21, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.777836


In [ ]:
model = cb.CatBoostClassifier(iterations=400, depth=15, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.782563


In [ ]:
model = KNeighborsClassifier(n_neighbors=11, metric="jaccard")
model.fit(X_train_fs, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.759979


## Usando 500 features

In [ ]:
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 500)

In [ ]:
model = xgb.XGBClassifier(n_estimators=400, objective='binary:logistic', max_depth=9, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.3, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794643


In [ ]:
model = lgb.LGBMClassifier(num_leaves=7, objective="binary", colsample_bytree=0.5, subsample=1, max_depth=7, n_jobs=1,
                              n_estimators=400, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.5,
               importance_type='split', learning_rate=0.1, max_depth=7,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=1, num_leaves=7, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.789391


In [ ]:
model = cb.CatBoostClassifier(iterations=500, depth=9, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.794118


In [ ]:
model = KNeighborsClassifier(n_neighbors=11, metric="jaccard")
model.fit(X_train_fs, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.761555


## Usando 1000 features

In [ ]:
X_train_fs, X_test_fs = select_features(X_train, y_train, X_test, chi2, 1000)

In [ ]:
model = xgb.XGBClassifier(n_estimators=500, objective='binary:logistic', max_depth=13, learning_rate=0.1,
                          subsample=1, colsample_bytree=0.5, n_jobs=1)
model.fit(X_train_fs, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=13,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.805672


In [ ]:
model = lgb.LGBMClassifier(num_leaves=14, objective="binary", colsample_bytree=0.7, subsample=1, max_depth=13, n_jobs=1,
                              n_estimators=500, min_child_samples=10, learning_rate=0.1)
model.fit(X_train_fs, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
               importance_type='split', learning_rate=0.1, max_depth=13,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=500, n_jobs=1, num_leaves=14, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1, subsample_for_bin=200000, subsample_freq=0)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.804097


In [ ]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.784898


In [ ]:
kfold = KFold(n_splits=6)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.787881


In [ ]:
kfold = KFold(n_splits=8)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.786828


In [ ]:
model = cb.CatBoostClassifier(iterations=500, depth=11, learning_rate=0.1)
model.fit(X_train_fs, y_train, verbose=False)

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.798319


In [ ]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

0:	learn: 0.6817894	total: 56.3ms	remaining: 28.1s
1:	learn: 0.6712211	total: 106ms	remaining: 26.3s
2:	learn: 0.6611519	total: 154ms	remaining: 25.5s
3:	learn: 0.6530371	total: 203ms	remaining: 25.1s
4:	learn: 0.6465166	total: 251ms	remaining: 24.8s
5:	learn: 0.6416743	total: 307ms	remaining: 25.3s
6:	learn: 0.6374526	total: 356ms	remaining: 25.1s
7:	learn: 0.6304438	total: 410ms	remaining: 25.2s
8:	learn: 0.6259872	total: 459ms	remaining: 25s
9:	learn: 0.6211229	total: 508ms	remaining: 24.9s
10:	learn: 0.6175116	total: 558ms	remaining: 24.8s
11:	learn: 0.6137663	total: 607ms	remaining: 24.7s
12:	learn: 0.6102923	total: 655ms	remaining: 24.6s
13:	learn: 0.6054569	total: 704ms	remaining: 24.4s
14:	learn: 0.6015143	total: 759ms	remaining: 24.5s
15:	learn: 0.5985845	total: 811ms	remaining: 24.5s
16:	learn: 0.5952284	total: 859ms	remaining: 24.4s
17:	learn: 0.5925807	total: 908ms	remaining: 24.3s
18:	learn: 0.5884456	total: 956ms	remaining: 24.2s
19:	learn: 0.5851178	total: 1.01s	remainin

In [ ]:
kfold = KFold(n_splits=6)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

0:	learn: 0.6803697	total: 65.6ms	remaining: 32.7s
1:	learn: 0.6702809	total: 115ms	remaining: 28.6s
2:	learn: 0.6627354	total: 165ms	remaining: 27.3s
3:	learn: 0.6564215	total: 220ms	remaining: 27.2s
4:	learn: 0.6496678	total: 281ms	remaining: 27.8s
5:	learn: 0.6440951	total: 330ms	remaining: 27.2s
6:	learn: 0.6381556	total: 380ms	remaining: 26.8s
7:	learn: 0.6334739	total: 432ms	remaining: 26.6s
8:	learn: 0.6294415	total: 481ms	remaining: 26.3s
9:	learn: 0.6237794	total: 532ms	remaining: 26.1s
10:	learn: 0.6201815	total: 582ms	remaining: 25.9s
11:	learn: 0.6170540	total: 631ms	remaining: 25.7s
12:	learn: 0.6124387	total: 680ms	remaining: 25.5s
13:	learn: 0.6079486	total: 735ms	remaining: 25.5s
14:	learn: 0.6043318	total: 784ms	remaining: 25.4s
15:	learn: 0.6011862	total: 833ms	remaining: 25.2s
16:	learn: 0.5980043	total: 885ms	remaining: 25.1s
17:	learn: 0.5939865	total: 940ms	remaining: 25.2s
18:	learn: 0.5902002	total: 989ms	remaining: 25s
19:	learn: 0.5878409	total: 1.04s	remainin

In [ ]:
kfold = KFold(n_splits=8)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

0:	learn: 0.6832246	total: 49.9ms	remaining: 24.9s
1:	learn: 0.6749472	total: 100ms	remaining: 25s
2:	learn: 0.6653723	total: 154ms	remaining: 25.5s
3:	learn: 0.6557400	total: 204ms	remaining: 25.3s
4:	learn: 0.6491003	total: 257ms	remaining: 25.4s
5:	learn: 0.6426752	total: 310ms	remaining: 25.5s
6:	learn: 0.6346087	total: 360ms	remaining: 25.3s
7:	learn: 0.6292011	total: 410ms	remaining: 25.2s
8:	learn: 0.6251417	total: 463ms	remaining: 25.3s
9:	learn: 0.6202204	total: 513ms	remaining: 25.2s
10:	learn: 0.6140753	total: 571ms	remaining: 25.4s
11:	learn: 0.6111639	total: 621ms	remaining: 25.3s
12:	learn: 0.6080425	total: 675ms	remaining: 25.3s
13:	learn: 0.6033001	total: 725ms	remaining: 25.2s
14:	learn: 0.5999496	total: 775ms	remaining: 25.1s
15:	learn: 0.5972803	total: 804ms	remaining: 24.3s
16:	learn: 0.5942792	total: 854ms	remaining: 24.3s
17:	learn: 0.5915259	total: 906ms	remaining: 24.3s
18:	learn: 0.5881227	total: 957ms	remaining: 24.2s
19:	learn: 0.5839728	total: 1.01s	remainin

In [ ]:
model = KNeighborsClassifier(n_neighbors=11, metric="jaccard")
model.fit(X_train_fs, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='jaccard',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform')

In [ ]:
y_test_hat = model.predict(X_test_fs)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.776786


In [ ]:
kfold = KFold(n_splits=8)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.780874


In [ ]:
kfold = KFold(n_splits=4)
resultados = cross_val_score(model, X_train_fs, y_train, cv=kfold)
print("Accuracy: %f" % (resultados.mean()))

Accuracy: 0.772115


## CatBoost

In [28]:
filter_bow_train = df_bow_train.loc[:,(df_bow_train.sum()>1)]
print(filter_bow_train.shape)
filter_list = filter_bow_train.columns.tolist()
filter_bow_test = df_bow_test.filter(items=filter_list)
print(filter_bow_train.shape)

(7613, 5925)
(7613, 5925)


In [84]:
train_bow_hash = pd.concat([train_hash,filter_bow_train], axis="columns")
print(train_bow_hash.shape)
test_bow_hash = pd.concat([test_hash,filter_bow_test], axis="columns")
print(test_bow_hash.shape)

(7613, 2451)
(3263, 2450)


In [78]:
X = train_bow_hash.drop(["id_original","keyword_original","location_original","text_original","target_label",
                      "clean_text"], axis=1)
y = train_bow_hash["target_label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5709, 1303)
(5709,)
(1904, 1303)
(1904,)


In [53]:
model = cb.CatBoostClassifier(iterations=700, depth=13, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

KeyboardInterrupt: ignored

In [38]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.783088


In [83]:
filter_bow_train = df_bow_train.loc[:,(df_bow_train.sum()>=5)]
print(filter_bow_train.shape)
filter_list = filter_bow_train.columns.tolist()
filter_bow_test = df_bow_test.filter(items=filter_list)
print(filter_bow_test.shape)

(7613, 2437)
(3263, 2437)


In [74]:
model = cb.CatBoostClassifier(iterations=500, depth=13, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

In [75]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.790966


In [76]:
filter_bow_train = df_bow_train.loc[:,(df_bow_train.sum()>=10)]
print(filter_bow_train.shape)
filter_list = filter_bow_train.columns.tolist()
filter_bow_test = df_bow_test.filter(items=filter_list)
print(filter_bow_train.shape)

(7613, 1295)
(7613, 1295)


In [81]:
model = cb.CatBoostClassifier(iterations=500, depth=11, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

In [82]:
y_test_hat = model.predict(X_test)
print("Accuracy score: %f" % (accuracy_score(y_test, y_test_hat)))

Accuracy score: 0.780462


In [85]:
# con 2450 features
X_train = train_bow_hash.drop(["id_original","keyword_original","location_original","text_original","target_label","clean_text"], axis=1)
X_test = test_bow_hash.drop(["id_original","keyword_original","location_original","text_original","clean_text"], axis=1)
y_train = train_bow_hash["target_label"]

In [86]:
model = cb.CatBoostClassifier(iterations=500, depth=13, learning_rate=0.1)
model.fit(X_train, y_train, verbose=False)

In [87]:
y_pred = model.predict(X_test)
y_pred

array([1, 1, 1, ..., 1, 1, 0])

In [88]:
test["target"] = y_pred
test[["id_original","target"]]

,id_original,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [90]:
from google.colab import files

test[["id_original","target"]].rename(columns={"id_original":"id"}).to_csv("pred13_catboost_bowhash", index=False)
files.download("pred13_catboost_bowhash")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>